In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from plotly import graph_objs as go
from plotly.offline import iplot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import accuracy_score, classification_report, f1_score, confusion_matrix
import numpy as np
from DecisionTree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [ ]:
df= pd.read_csv('heart.csv')
df

# Exploring Data


In [ ]:
print("Initial rows in in dataset:")
df.head()

In [ ]:
# Get the dimensions of the Dataset
print("Dimensions of the Dataset (Rows, Columns):")
df.shape

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
print("Statistical summary of numeric features:")
df.describe()

In [ ]:
print("Statistical summary of categorical features:")
df.describe(include=object)

In [ ]:
# Removing any leading, and trailing whitespaces in columns
df.columns = df.columns.str.strip()

In [ ]:
df.columns.tolist()

In [ ]:
# Check nulls
null = df.isna().sum()
total = len(df)
percentage = (null / total) * 100
print(percentage[percentage != 0])
print(f"The percentage of nulls in each column:\n{percentage}")

# Preprocessing

In [ ]:
df['RestingBP'].replace(0, np.nan, inplace=True)
df['Cholesterol'].replace(0, np.nan, inplace=True)

In [ ]:
# Check nulls
null = df.isna().sum()
total = len(df)
percentage = (null / total) * 100
print(percentage[percentage != 0])
print(f"The percentage of nulls in each column:\n{percentage}")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import KNNImputer

# Use KNN imputation
imputer = KNNImputer(n_neighbors=5)
df[['RestingBP', 'Cholesterol']] = imputer.fit_transform(df[['RestingBP', 'Cholesterol']])

In [ ]:
# A function to the visualization of numeric & categorical features with the target feature
def num_features(df, num_var, tar_var):
    x0 = df[df[tar_var]==0][num_var]
    x1 = df[df[tar_var]==1][num_var]

    trace1 = go.Histogram(x = x0,
                               name = "No",
                               opacity = 0.75,
                               marker=dict(color="#0A4D68",
                                           line = dict(color = '#DBE6EC',
                                                       width = 1)))

    trace2 = go.Histogram(x = x1,
                               name = "Yes",
                               opacity = 0.75,
                               marker=dict(color="#A6D0DD",
                                           line = dict(color = '#DBE6EC',
                                                       width = 1)))

    data = [trace1, trace2]

    layout = go.Layout(title={'text': num_var,
                         'y':0.9,
                         'x':0.5,
                         'xanchor':'center',
                         'yanchor':'top'},
                         barmode='overlay',
                         yaxis=dict(title='Count'),
                         template = 'plotly_dark')

    fig = go.Figure(data=data, layout=layout)

    iplot(fig)

In [ ]:
df['HeartDisease'].value_counts()

In [ ]:
df['Cholesterol'].value_counts()

In [ ]:
for i in df.describe().columns:
  num_features(df, i, "HeartDisease")

In [ ]:
# Separating Numeric and Categorical Columns
numeric_col = [col for col in df.columns if df[col].dtype != 'object']
categorical_col = [col for col in df.columns if df[col].dtype == 'object']

In [ ]:
# Visualize Categorical features
for feature in categorical_col:
    num_features(df, feature, "HeartDisease")
    print("\n {}".format(df[feature].value_counts()))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# Outliers by boxplots
for col in numeric_col:
   plt.figure(figsize=(8, 6))
   sns.boxplot(data= df[col])
   plt.title(f'{col} Box plot')
   #plt.show()

In [ ]:
corr_matrix = df[numeric_col].corr()
plt.figure(figsize=(20, 15))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
le = LabelEncoder()
for column in df.select_dtypes(include=['object']):
    df[column] = le.fit_transform(df[column])

In [ ]:
X=df.drop(['HeartDisease'],axis=1)
Y=df["HeartDisease"] # Target

print(f"the shape of the inputs x is: {X.shape}")
print(f"the shape of the targets y is: {Y.shape}")

In [ ]:
Y = np.where(Y == 0, -1, 1)  

# Split dataset: 60% train, 20% validation, 20% test
X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=2/3, random_state=42)

# Models

#### Trying with multible scalers

In [ ]:
# Define scalers
scalers = {
    "StandardScaler": StandardScaler(),
    "MinMaxScaler": MinMaxScaler(),
    "RobustScaler": RobustScaler()
}

# Decsision Tree

In [ ]:
# Iterate over different scalers
for name, scaler in scalers.items():
    print(f"\n=== Using {name} ===")
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    # Train AdaBoost model
    model = DecisionTreeClassifier(max_depth=100)
    model.fit(X_train_scaled, y_train)
    
    # Predict on validation set
    y_val_pred = model.predict(X_val_scaled)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_f1 = f1_score(y_val, y_val_pred)
    
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Validation F1-Score: {val_f1:.4f}")
    
    # Final prediction on test set
    y_test_pred = model.predict(X_test_scaled)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test F1-Score: {test_f1:.4f}")
    
    # Confusion matrix
    conf_matrix = confusion_matrix(y_test, y_test_pred)
    print("Confusion Matrix:")
    print(conf_matrix)
    
    plt.figure(figsize=(6, 4))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=[-1, 1], yticklabels=[-1, 1])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix ({name})")
    plt.show()

# Bagging

In [ ]:
from collections import Counter

class Bagging:
    def __init__(self, n_estimators=10):
        self.n_estimators = n_estimators
        self.models = []

    def bootstrap_sample(self, X, y):
        n_samples = len(X)
        indices = np.random.choice(n_samples, n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        self.models = []
        for _ in range(self.n_estimators):
            tree = DecisionTreeClassifier()  # Base model
            X_sample, y_sample = self.bootstrap_sample(X, y)
            tree.fit(X_sample, y_sample)
            self.models.append(tree)

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.models])
        majority_votes = np.apply_along_axis(lambda x: Counter(x).most_common(1)[0][0], axis=0, arr=predictions)
        return majority_votes


# Iterate over different scalers
for name, scaler in scalers.items():
    print(f"\n=== Using {name} ===")

    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    # Train Bagging model
    model = Bagging(n_estimators=10)
    model.fit(X_train_scaled, y_train)

    # Predict on validation set
    y_val_pred = model.predict(X_val_scaled)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_f1 = f1_score(y_val, y_val_pred)

    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Validation F1-Score: {val_f1:.4f}")

    # Final prediction on test set
    y_test_pred = model.predict(X_test_scaled)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)

    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test F1-Score: {test_f1:.4f}")

    # Confusion matrix
    conf_matrix = confusion_matrix(y_test, y_test_pred)
    print("Confusion Matrix:")
    print(conf_matrix)

    # Plot Confusion Matrix
    plt.figure(figsize=(6, 4))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=[0, 1], yticklabels=[0, 1])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix ({name})")
    plt.show()


# Ada Boost

In [ ]:

class AdaBoost:
    def __init__(self, n_estimators=50):
        self.n_estimators = n_estimators
        self.alphas = []
        self.models = []
    
    def fit(self, X, Y):
        n_samples, n_features = X.shape
        w = np.ones(n_samples) / n_samples  # Initialize sample weights
        
        for _ in range(self.n_estimators):
            stump = DecisionTreeClassifier(max_depth=1)
            stump.fit(X, Y, sample_weight=w)
            y_pred = stump.predict(X)

            err = np.sum(w * (y_pred != Y)) / np.sum(w)  # Compute error
            alpha = 0.5 * np.log((1 - err) / (err + 1e-10))  # Model weight
            
            w *= np.exp(-alpha * Y * y_pred)  # Update sample weights
            w /= np.sum(w)  # Normalize
            
            self.models.append(stump)
            self.alphas.append(alpha)
    
    def predict(self, X):
        weak_preds = np.array([alpha * model.predict(X) for model, alpha in zip(self.models, self.alphas)])
        return np.sign(np.sum(weak_preds, axis=0))  # Aggregate predictions

# Iterate over different scalers
for name, scaler in scalers.items():
    print(f"\n=== Using {name} ===")
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    # Train AdaBoost model
    model = AdaBoost(n_estimators=50)
    model.fit(X_train_scaled, y_train)
    
    # Predict on validation set
    y_val_pred = model.predict(X_val_scaled)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_f1 = f1_score(y_val, y_val_pred)
    
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Validation F1-Score: {val_f1:.4f}")
    
    # Final prediction on test set
    y_test_pred = model.predict(X_test_scaled)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print(f"Test F1-Score: {test_f1:.4f}")
    
    # Confusion matrix
    conf_matrix = confusion_matrix(y_test, y_test_pred)
    print("Confusion Matrix:")
    print(conf_matrix)
    
    plt.figure(figsize=(6, 4))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=[-1, 1], yticklabels=[-1, 1])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix ({name})")
    plt.show()
    

# KNN

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Split data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Define different scalers
scalers = {
    "StandardScaler": StandardScaler(),
    "MinMaxScaler": MinMaxScaler(),
    "RobustScaler": RobustScaler()
}

# Define hyperparameter grid
param_grid = {
    "n_neighbors": [3, 5, 7, 11],  # Different values of k
    "metric": ["euclidean", "manhattan"]  # Distance metrics
}

best_model = None
best_f1 = 0
best_scaler_name = None

# Iterate over different scalers
for scaler_name, scaler in scalers.items():
    # Scale the data
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    # Hyperparameter tuning with GridSearchCV
    grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring="f1_weighted", n_jobs=-1)
    grid_search.fit(X_train_scaled, y_train)

    # Evaluate on validation set
    best_knn = grid_search.best_estimator_
    y_val_pred = best_knn.predict(X_val_scaled)
    f1 = f1_score(y_val, y_val_pred, average="weighted")

    print(f"Scaler: {scaler_name} | Best k: {grid_search.best_params_['n_neighbors']} | Best Metric: {grid_search.best_params_['metric']} | F1 Score: {f1:.4f}")

    # Store the best model
    if f1 > best_f1:
        best_f1 = f1
        best_model = best_knn
        best_scaler_name = scaler_name
        best_scaler = scaler  # Save the best scaler

# Apply best scaler to test data
X_test_scaled = best_scaler.transform(X_test)

# Evaluate on test set
y_test_pred = best_model.predict(X_test_scaled)

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred, average="weighted")  # Change to 'macro' or 'binary' as needed
conf_matrix = confusion_matrix(y_test, y_test_pred)

# Print final results
print("\nBest Model Performance on Test Set:")
print(f"Best Scaler: {best_scaler_name}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1 Score: {f1:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_test_pred))

# Plot confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", xticklabels=np.unique(Y), yticklabels=np.unique(Y))
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Define different scalers
scalers = {
    "StandardScaler": StandardScaler(),
    "MinMaxScaler": MinMaxScaler(),
    "RobustScaler": RobustScaler()
}

# Define hyperparameter grid for logistic regression
param_grid = {"C": [0.001, 0.01, 0.1, 1, 10, 100]}

best_model = None
best_f1 = 0
best_scaler_name = None

# Iterate over different scalers
for scaler_name, scaler in scalers.items():
    # Scale the data
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # Hyperparameter tuning with GridSearchCV
    grid_search = GridSearchCV(LogisticRegression(max_iter=1000, random_state=42), param_grid, cv=5, scoring="f1_weighted", n_jobs=-1)
    grid_search.fit(X_train_scaled, y_train)

    # Evaluate on validation set
    best_log_reg = grid_search.best_estimator_
    y_val_pred = best_log_reg.predict(X_val_scaled)
    f1 = f1_score(y_val, y_val_pred, average="weighted")

    print(f"Scaler: {scaler_name} | Best C: {grid_search.best_params_['C']} | F1 Score: {f1:.4f}")

    # Store the best model
    if f1 > best_f1:
        best_f1 = f1
        best_model = best_log_reg
        best_scaler_name = scaler_name
        best_scaler = scaler  # Save the best scaler

# Apply best scaler to test data
X_test_scaled = best_scaler.transform(X_test)

# Evaluate on test set
y_test_pred = best_model.predict(X_test_scaled)

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred, average="weighted")  # Change to 'macro' or 'binary' as needed
conf_matrix = confusion_matrix(y_test, y_test_pred)

# Print final results
print("\nBest Model Performance on Test Set:")
print(f"Best Scaler: {best_scaler_name}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1 Score: {f1:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_test_pred))

# Plot confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", xticklabels=np.unique(Y), yticklabels=np.unique(Y))
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# FNN

In [ ]:


# Split data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=2/3, random_state=42)

# Define different scalers
scalers = {
    "StandardScaler": StandardScaler(),
    "MinMaxScaler": MinMaxScaler(),
    "RobustScaler": RobustScaler()
}

# Define hyperparameter options
hidden_layer_sizes = [16, 32]
learning_rates = [0.001, 0.01]
epochs_list = [50, 100]

best_model = None
best_f1 = 0
best_scaler_name = None
best_hparams = {}

# Iterate over different scalers
for scaler_name, scaler in scalers.items():
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # Iterate over hyperparameter combinations
    for hidden_size in hidden_layer_sizes:
        for lr in learning_rates:
            for epochs in epochs_list:
                # Build neural network model
                model = keras.Sequential([
                    keras.layers.Dense(hidden_size, activation="relu", input_shape=(X_train_scaled.shape[1],)),
                    keras.layers.Dense(1, activation="sigmoid")
                ])

                # Compile the model
                model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                              loss="binary_crossentropy",
                              metrics=["accuracy"])

                # Train the model
                history = model.fit(X_train_scaled, y_train, validation_data=(X_val_scaled, y_val),
                                    epochs=epochs, batch_size=32, verbose=0)

                # Predict on validation set
                y_val_pred_probs = model.predict(X_val_scaled)
                y_val_pred = (y_val_pred_probs > 0.5).astype(int)

                # Compute F1-score
                f1 = f1_score(y_val, y_val_pred, average="weighted")

                print(f"Scaler: {scaler_name} | Hidden Layer: {hidden_size} | LR: {lr} | Epochs: {epochs} | F1: {f1:.4f}")

                # Store the best model
                if f1 > best_f1:
                    best_f1 = f1
                    best_model = model
                    best_scaler_name = scaler_name
                    best_scaler = scaler
                    best_hparams = {"hidden_layer": hidden_size, "learning_rate": lr, "epochs": epochs}

# Apply best scaler to test data
X_test_scaled = best_scaler.transform(X_test)

# Evaluate on test set
y_test_pred_probs = best_model.predict(X_test_scaled)
y_test_pred = (y_test_pred_probs > 0.5).astype(int)

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred, average="weighted")
conf_matrix = confusion_matrix(y_test, y_test_pred)

# Print results
print("\nBest Model Performance on Test Set:")
print(f"Best Scaler: {best_scaler_name}")
print(f"Best Hyperparameters: {best_hparams}")
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1 Score: {f1:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_test_pred))

# Plot confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", xticklabels=np.unique(Y), yticklabels=np.unique(Y))
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()
